In [1]:
import os
import glob
import time
import json
from tqdm.notebook import tqdm

from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def get_xpath(element):
    xpath_script = """
    function getPathTo(element) {
        // if (element.id !== '')
        //     return 'id(\"'+element.id+'\")';
        if (element === document.body)
            return element.tagName;
        var ix= 0;
        var siblings= element.parentNode.childNodes;
        for (var i= 0; i<siblings.length; i++) {
            var sibling= siblings[i];
            if (sibling===element)
                return getPathTo(element.parentNode) + '/' + element.tagName + '[' + (ix + 1) + ']';
            if (sibling.nodeType===1 && sibling.tagName===element.tagName)
                ix++;
        }
    }
    const path = getPathTo(arguments[0]);
    if (path.startsWith('id(')) {
        return path;
    }
    return '//' + path;
    """
    return driver.execute_script(xpath_script, element)

In [3]:
def create_application_directory(app_name):
    if os.path.exists(app_name):
        return False
    os.makedirs(app_name)
    return True


def get_next_index(directory):
    if len(os.listdir(directory)) == 0:
        return 1
    return sorted(list(map(lambda x: int(x.split('.')[0]), os.listdir(directory))))[-1] + 1


def take_full_screenshot(element, path):
    # store original size
    original_size = driver.get_window_size()
    
    required_height = element.size['height']
    
    # scroll to element
    driver.execute_script("return arguments[0].scrollIntoView();", element)
    time.sleep(0.5)
    # set window size
    # driver.set_window_size(original_size['width'], required_height + 128)
    
    # now that the element is within the viewport, take screenshot of element
    element.screenshot(path)

    # return window to original size
    # driver.set_window_size(original_size['width'], original_size['height'])


def add_entry_to_directory(element, app_name):
    create_application_directory(app_name)
    index = get_next_index(app_name)
    
    element_html = element.get_attribute('outerHTML')
    pretty_html = bs(element_html, 'html.parser').prettify()

    with open(os.path.join(app_name, f'{index}.json'), 'w+') as f:
        json.dump({
            'url': driver.current_url,
            'form_xpath': get_xpath(element)
        }, f)
        f.close()
    
    with open(os.path.join(app_name, f'{index}.html'), 'w+') as f:
        f.write(pretty_html)
        f.close()
    
    take_full_screenshot(element, os.path.join(app_name, f'{index}.png'))

In [4]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--window-size=3072x1920");

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [5]:
URL = "https://www.aircanada.com/ca/en/aco/home.html"
APP_NAME = 'aircanada'
driver.get(URL)

In [6]:
driver.maximize_window()

In [14]:
forms = driver.find_elements(By.TAG_NAME, 'form')

print('forms:', len(forms))

for i, form in enumerate(forms):
    print('form', i + 1)
    try:
        add_entry_to_directory(form, APP_NAME)
    except Exception as e:
        print(e)

forms: 2
form 1
form 2
